# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [ ]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster

try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)


#### Create Keyspace

In [ ]:
# Create a Keyspace 

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [ ]:
# Set KEYSPACE to the keyspace specified above

try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Create tables for the 3 queries

In [ ]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# fields required in select statement - artist, song title, song length
# keys required for where criteria - sessionId, itemInSession

query = """CREATE TABLE IF NOT EXISTS song_by_session_item 
            (sessionId INT, itemInSession INT, artist TEXT, song TEXT, length DECIMAL,
            PRIMARY KEY (sessionId, itemInSession))"""
try:
    session.execute(query)
except Exception as e:
    print(e)
    

In [ ]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# fields required in select statement - artist name, song, first name, last name
# keys required for where and sorting criteria 
# - primary - for condition and so it is unique - userid, sessionid
# - clustering key, for sort criteria - iteminsession 

query = """CREATE TABLE IF NOT EXISTS song_by_user_session 
            (userId INT, sessionId INT, itemInSession INT, artist TEXT, song TEXT, username TEXT,
            PRIMARY KEY ((userId, sessionId), itemInSession))"""
try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# fields required in select statement - user name (first name + last name)
# keys required - song (for query criteria), user id (to make key unique - this is assuming 
#  that a user hasn't listened to the same song multiple times in different sessions )

query = """CREATE TABLE IF NOT EXISTS user_for_song 
            (song TEXT, userId INT, username TEXT, PRIMARY KEY (song, userId))"""
try:
    session.execute(query)
except Exception as e:
    print(e)

## Read through CSV file and load data to all 3 tables

### Column references for fields in inserts below

line[0] = 'artist' text<br>
line[1] = 'firstName' text, combined with lastName for username<br>
line[2] = 'gender' text<br>
line[3] = 'itemInSession' convert to int<br>
line[4] = 'lastName' text, combined with firstName for username<br>
line[5] = 'length' convert to float<br>
line[6] = 'level' text<br>
line[7] = 'location' text<br>
line[8] = 'sessionId' convert to int<br>
line[9] = 'song' text<br>
line[10] = 'userId' convert to int<br>


In [ ]:
file = 'event_datafile_new.csv'

query1 = """INSERT INTO song_by_session_item (sessionId, itemInSession, artist, song, length) 
            VALUES (%s, %s, %s, %s, %s)"""
query2 = """INSERT INTO song_by_user_session (userId, sessionId, itemInSession, artist, song, username)
            VALUES (%s, %s, %s, %s, %s, %s)"""
query3 = """INSERT INTO user_for_song (song, userId, username)
            VALUES (%s, %s, %s)"""

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # All fields in csv file are quoted, so string format. Convert required fields to numeric,
        # in try clause to catch the ValueError if there's data that can't be converted.
        try:
            itemInSession_int = int(line[3])
            length_float = float(line[5])
            sessionId_int = int(line[8])
            userId_int = int(line[10])
        except ValueError as e:
            print(e)
            print("Error converting data from csv file to numeric format")
            break # from for loop
            
        # database inserts
        try:
            session.execute(query1, (sessionId_int, itemInSession_int, line[0], line[9], length_float))
            session.execute(query2, (userId_int, sessionId_int, itemInSession_int, line[0], line[9], f"{line[1]} {line[4]}"))
            session.execute(query3, (line[9], userId_int, f"{line[1]} {line[4]}"))
        except Exception as e:
            print(e)
            break # from for loop


#### Code used so the result sets from the Cassandra query can be converted to a pandas dataframe for a nice display of the result.

##### code from: https://stackoverflow.com/questions/41247345/python-read-cassandra-data-into-pandas

In [ ]:
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

session.row_factory = pandas_factory
session.default_fetch_size = None

#### SELECT statements to verify that the data has been inserted into each table and to get the requested information

In [ ]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query1 = """
SELECT artist, song, length FROM song_by_session_item
WHERE sessionId = 338 AND itemInSession = 4"""
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

# next line of code also from Stack Overflow link referenced above
df = rows._current_rows
print("Query 1:  Give me the artist, song title and song's length in the music app history")
print("that was heard during sessionId = 338, and itemInSession = 4")
print(query1 + '\n')
df

In [ ]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query2 = """
SELECT artist, song, username FROM song_by_user_session
WHERE userId = 10 AND sessionId = 182"""
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
# next line of code also from Stack Overflow link referenced above
df = rows._current_rows
print("Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)")
print("for userid = 10, sessionid = 182")
print(query1 + '\n')
df

In [ ]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# select - first name, last name
# keys - song (for query), session id, item in session (both to make key unique)

query3 = """
SELECT username FROM user_for_song
WHERE song = 'All Hands Against His Own'"""
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
# next line of code also from Stack Overflow link referenced above
df = rows._current_rows
print("Query 3: Give me every user name (first and last) in my music app history ")
print("who listened to the song 'All Hands Against His Own'")
print(query1 + '\n')
df

### Drop the tables before closing out the sessions

In [ ]:
## Drop the table before closing out the sessions

In [ ]:
try:
    session.execute("DROP TABLE song_by_session_item")
    session.execute("DROP TABLE song_by_user_session")
    session.execute("DROP TABLE user_for_song")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()